## Парсим Stepik

Наша цель — получить следующую информацию о курсах на сайте stepik.org:

- Название курса
- Краткое описание
- Навыки (Чему вы научитесь)
- Количество студентов
- Цена

In [17]:
import requests                         
import pandas as pd                     
import time                             
from bs4 import BeautifulSoup           
from fake_useragent import UserAgent    
from tqdm.auto import tqdm          
import warnings                         
warnings.filterwarnings("ignore")

In [ ]:
def getPageLinks(page_number):
    
    page_link = 'http://stepik.org/catalog/{}'.format(page_number)

    response = requests.get(page_link, headers={'User-Agent': UserAgent().safari})

    if not response.ok:
        return []

    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['course-card__title'])

    links = ['http://stepik.org' + link.attrs['href'] + '/promo' for link in links]

    return links

In [ ]:
def getData(page):
    
    response = requests.get(page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        return response.status_code

    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    main_content_block = soup.find('main', attrs={'class':'main-content'})
    title = main_content_block.find('h1')
    title = "" if not title else title.text.strip()

    short_description = (main_content_block
                         .find('div',
                               attrs={'class':'shortened-text ember-view'})
                         .text.strip())

    aquired_skills = (main_content_block
                      .find('section',
                            attrs={'class':"course-promo__content-block" })
                      .text.strip())

    students = soup.find('div', attrs={'class':'course-promo-summary__students'})
    students = None if not students else int(students.text.strip().split()[0].replace(',', ''))

    price = (main_content_block
             .find('span',
                   attrs={'class':"format-price" })
             .text.strip())

    return {'title': title,
            'about':short_description,
            'skills':aquired_skills,
            'students':students,
            'price':price}

In [ ]:
getData('https://stepik.org/course/184930/promo')

{'title': 'Основы PHP: Структура и Синтаксис',
 'about': 'Курс сосредоточен на PHP7.4\nЭтот курс позволит вам понять потенциал PHP как языка программирования в WEB-разработке. Основное внимание уделяется практической применимости в бизнес-процессах, подкрепленное реальными бизнес-примерами.',
 'skills': 'Чему вы научитесь\n\nРешать задачи в PHP.\nПонимать, что требуется бизнесу от начинающего программиста.\nРаботать с синтаксисом PHP.\nСоблюдать стандарты кодирования.\nИспользовать условные конструкции.\nИспользовать циклы.\nСоздавать скрипты на PHP.\nСоздавать, читать и обрабатывать файлы.',
 'students': 76,
 'price': '700\xa0₽'}

In [ ]:
links = getPageLinks(1)
final_df = pd.DataFrame(columns=['title', 'about', 'skills', 'students', 'price'])
for link in tqdm(links):
    try:
        data_row = getData(link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

In [ ]:
final_df = pd.DataFrame(columns=['title', 'about', 'skills', 'students', 'price'])

for page_number in tqdm(range(500), desc='Pages'):
    links = getPageLinks(page_number)
    for link in tqdm(links, desc='Courses', leave=False):
        for i in range(3):
            try:
                data_row = getData(link)
                final_df = final_df.append(data_row, ignore_index=True)
            except:
                continue
            time.sleep(0.34)

In [ ]:
final_df = final_df.drop_duplicates().dropna()
len(final_df)

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('../data/stepik_parsed.csv', index=False)